# Importing Dataset

In [1]:
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle as pk
import json

In [2]:
df = pl.read_csv('./data/final_dataset.csv')
df.head()

sentiment,review
i64,str
1,"""I'm reading a lot of reviews s…"
1,"""This soundtrack is my favorite…"
1,"""I truly like this soundtrack a…"
1,"""If you've played the game, you…"
1,"""I am quite sure any of you act…"


# Preprocessing

In [3]:
import re
from nltk.corpus import stopwords

In [4]:
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

In [5]:
def preprocess_text(text):
    # remove tags and convert to lowercase
    sentence = remove_tags(text).lower()

    # remove punctuation, numbers, and single characters, and collapse multiple spaces
    sentence = re.sub(r'[^a-z\s]', ' ', sentence)  # remove non-alphabetic characters
    sentence = re.sub(r'\b[a-z]\b', '', sentence)  # remove single characters
    sentence = re.sub(r'\s+', ' ', sentence).strip()  # collapse multiple spaces

    # remove stopwords
    stop_words = set(stopwords.words('english'))
    sentence = ' '.join([word for word in sentence.split() if word not in stop_words])
    
    return sentence


In [6]:
# X = []
# sentences = list(df['review'])

# for sen in sentences:
#     X.append(preprocess_text(sen))
    
# with open('./data/preprocessed_text.txt', 'wb') as f:
#     pk.dump(X, f)

In [7]:
with open('./data/preprocessed_text.txt', 'rb') as f:
    X = pk.load(f)

In [8]:
X[2]

'truly like soundtrack enjoy video game music played game music enjoy truly relaxing peaceful disk one favorites scars time life death forest illusion fortress ancient dragons lost fragment drowned valley disk two draggons galdorb home chronomantique prisoners fate gale girlfriend likes zelbessdisk three best three garden god chronopolis fates jellyfish sea burning orphange dragon prayer tower stars dragon god radical dreamers unstealable jewel overall excellent soundtrack brought like video game music xander cross'

In [9]:
from sklearn.model_selection import train_test_split

y = df['sentiment']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preparing Embedding Layer

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2024-09-16 11:14:59.656353: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-16 11:14:59.667810: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-16 11:14:59.680076: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-16 11:14:59.708077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-16 11:15:01.247149: W tensorflow/compiler/tf2

In [12]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train)

X_train = word_tokenizer.texts_to_sequences(X_train)
X_test = word_tokenizer.texts_to_sequences(X_test)

In [13]:
# tokenised_data = {
#     'X_train' : X_train,
#     'y_train' : y_train.to_list(),
#     'X_test' : X_test,
#     'y_test' : y_test.to_list()
# }

# with open('./data/tokenized_data.json', 'w') as f:
#     json.dump(tokenised_data, f)

In [14]:
# with open('./data/tokenized_data.json', 'r') as f:
#     tokenised_data = json.load(f)
    
# X_train = tokenised_data['X_train']
# X_test = tokenised_data['X_test']
# y_train = tokenised_data['y_train']
# y_test = tokenised_data['y_test']

In [15]:
# vocab_length = len(word_tokenizer.word_index) + 1
# vocab_length
vocab_length = 779656

In [16]:
maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [17]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [18]:
embeddings_dict = dict()

with open('./data/glove.6B.100d.txt',  'r') as glove_file:
    for line in glove_file:
        records = line.split()
        word = records[0]
        vector_dimentions = np.asarray(records[1:], dtype='float32')
        embeddings_dict[word] = vector_dimentions

In [19]:
# embeddings_dict

In [20]:
# with open('./data/word_tokenizer_items.txt', 'w') as f:
#     pk.dump(word_tokenizer.word_index.items(), f)

In [21]:
embedding_matrix = np.zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [22]:
type(embedding_matrix), embedding_matrix[1, :]

(numpy.ndarray,
 array([-1.97439998e-01,  4.48309988e-01,  1.36889994e-01, -1.55949995e-01,
         9.35999990e-01,  7.29860008e-01,  3.40990007e-01, -3.38959992e-01,
        -8.95690024e-02, -4.77059990e-01,  3.51119995e-01, -4.21979994e-01,
        -1.22210003e-01, -6.33750036e-02, -4.58200008e-01,  7.87230015e-01,
         9.40450013e-01,  8.11010003e-02, -2.32240006e-01,  4.07779992e-01,
         3.32580000e-01, -4.44579989e-01, -4.71170008e-01,  1.48519993e-01,
         9.63079989e-01, -6.52669966e-02, -5.36610000e-02, -6.74740016e-01,
        -4.23640013e-01,  9.43920016e-02, -3.86680007e-01,  1.82370007e-01,
        -1.28460005e-01, -2.19520003e-01, -5.89929998e-01,  7.36020029e-01,
        -2.40089998e-01,  3.23920012e-01, -2.46629998e-01, -4.06839997e-01,
        -5.24680018e-01,  4.61739987e-01, -1.49360001e-01, -1.19989999e-01,
        -1.39899999e-01, -4.49440002e-01, -2.65650004e-01, -7.00609982e-01,
         3.01880002e-01, -1.12089999e-01,  6.63230002e-01,  3.96979988e-

In [23]:
# with open('./data/embedding_matrix.txt', 'wb') as f:
#     pk.dump(embedding_matrix, f)

In [24]:
# with open('./data/embedding_matrix.txt', 'rb') as f:
#     emb_mat = pk.load(f)
    
# type(emb_mat), emb_mat[1, :]

# LSTM Model

In [25]:
# !pip install keras-tuner

In [33]:
import tensorflow as tf
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Dropout, Embedding, LSTM, GRU, GlobalMaxPooling1D, BatchNormalization
from keras.models import Sequential

In [27]:
def build_lstm_model(hp):
    model = Sequential([
        Embedding(vocab_length,
                  100,
                  weights=[embedding_matrix],
                  input_length=maxlen,
                  trainable=False),
        
        LSTM(hp.Int('lstm_units', min_value=64, max_value=256, step=32)),
        
        Dense(1, activation='sigmoid')
    ])


    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

    return model

In [28]:
tuner = kt.RandomSearch(
    build_lstm_model,            
    objective='val_accuracy',    
    max_trials=10,               
    executions_per_trial=1,      
    directory='./tuning_dir',   
    project_name='lstm_tuning'   
)

Reloading Tuner from ./tuning_dir/lstm_tuning/tuner0.json


I0000 00:00:1726465675.300605    7782 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726465675.465477    7782 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726465675.465733    7782 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726465675.468920    7782 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [29]:
# tuner.search(X_train_, y_train_, 
#              epochs=5, 
#              validation_data=(X_val, y_val),
#              batch_size=32)

In [30]:
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

2024-09-16 11:18:02.166118: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.
2024-09-16 11:18:03.200547: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 100)       │    77,965,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 192)            │       225,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 78,190,817 (298.27 MB)

 Trainable params: 225,217 (879.75 KB)

 Non-trainable params: 77,965,600 (297.42 MB)

In [35]:
with tf.device('/GPU:0'):  # Force the model to run on the GPU
    lstm_model = Sequential([
        Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False),
        
        LSTM(128, return_sequences=True),
        Dropout(0.3),
        
        LSTM(128, return_sequences=True),  # Keep the output 3D
        Dropout(0.3),
        
        GlobalMaxPooling1D(),  # Now works as it has a 3D input
        
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(1, activation='sigmoid')
    ])

    # Compile the model (ensure loss is compatible with mixed precision)
    lstm_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['acc']
    )

    # Train the model
    lstm_model_history = lstm_model.fit(
        X_train,
        y_train,
        batch_size=128,
        epochs=10,
        verbose=1,
        validation_data=(X_val, y_val)https://colab.research.google.com/drive/1xpAF-8nGvsbqXX4DvgmOPczaXASdMRVY?authuser=2#scrollTo=E09V-awqVow2
    )

2024-09-16 11:22:13.503271: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


Epoch 1/10


2024-09-16 11:22:14.150119: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1279999200 exceeds 10% of free system memory.
2024-09-16 11:22:17.585002: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


25000/25000 ━━━━━━━━━━━━━━━━━━━━ 382s 15ms/step - acc: 0.8481 - loss: 0.3446 - val_acc: 0.8737 - val_loss: 0.3097
Epoch 2/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 375s 15ms/step - acc: 0.8982 - loss: 0.2481 - val_acc: 0.9092 - val_loss: 0.2614
Epoch 3/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 377s 15ms/step - acc: 0.9068 - loss: 0.2302 - val_acc: 0.9127 - val_loss: 0.2451
Epoch 4/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 376s 15ms/step - acc: 0.9112 - loss: 0.2210 - val_acc: 0.9176 - val_loss: 0.2214
Epoch 5/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 375s 15ms/step - acc: 0.9139 - loss: 0.2151 - val_acc: 0.9197 - val_loss: 0.2238
Epoch 6/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 378s 15ms/step - acc: 0.9166 - loss: 0.2088 - val_acc: 0.9193 - val_loss: 0.2287
Epoch 7/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 379s 15ms/step - acc: 0.9182 - loss: 0.2053 - val_acc: 0.9226 - val_loss: 0.2171
Epoch 8/10
25000/25000 ━━━━━━━━━━━━━━━━━━━━ 379s 15ms/step - acc: 0.9196 - loss: 0.2020 - val_acc: 0.9243 - val_loss: 0.2176
Epoch 9/10


In [32]:
# lstm_model.compile(
#     optimizer='adam',
#     loss='binary_crossentropy',
#     metrics=['acc']
# )

In [33]:
score = best_model.evaluate(X_test, y_test, verbose=1)

print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

2024-09-15 19:21:23.520660: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 320000000 exceeds 10% of free system memory.
2024-09-15 19:21:24.454370: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


25000/25000 ━━━━━━━━━━━━━━━━━━━━ 52s 2ms/step - accuracy: 0.9044 - loss: 0.2341
Test Score : 0.23308660089969635
Test Accuracy: 0.9048100113868713


In [36]:
score = lstm_model.evaluate(X_test, y_test, verbose=1)

print(f'Test Score : {score[0]}\nTest Accuracy: {score[1]}')

25000/25000 ━━━━━━━━━━━━━━━━━━━━ 100s 4ms/step - acc: 0.9098 - loss: 0.2433
Test Score : 0.24265900254249573
Test Accuracy: 0.910147488117218


In [37]:
lstm_model.save('./models/lstm_model.h5')

In [32]:
# best_model.save('./models/Best_LSTM_model.h5')

2024-09-15 20:44:59.829545: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 311862400 exceeds 10% of free system memory.


In [33]:
# !pip install tensorflowjs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.7/780.7 kB 17.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 MB 18.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 23.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.6/289.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.7/269.7 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 22.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 16.3 MB/s eta 0

In [34]:
# !tensorflowjs_converter --input_format keras ./models/Best_LSTM_model.h5 ./models/tf_js_models

2024-09-15 20:47:22.793449: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-15 20:47:22.879356: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-15 20:47:22.902584: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-15 20:47:23.061961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-15 20:47:24.061751: W tensorflow/compiler/tf2